In [1]:
import pandas as pd
import tensorflow as tf
import scipy as sc
from scipy.spatial import distance
import cv2
import numpy as np
import threading
import time
from multiprocessing import Queue
import os
import json
from base64 import b64encode
from typing import (
    Any,
    List,
    Tuple,
)

import align.detect_face    # type: ignore
import facenet              # type: ignore

import matplotlib.pyplot as pl
%matplotlib inline

In [2]:
base_path = "D:/NNA/"
data_path = base_path + "data/"
facenet_path = base_path + "node-facenet/"
model_path = facenet_path + "models/"

minimum_size_of_face = 60

In [3]:
frames = Queue(7*2)
faces = Queue(7*2)

In [4]:
class MtcnnBridge():
    """
    MTCNN Face Alignment
    """
    def __init__(self) -> None:
        self.graph = self.session = None            # type: Any
        self.pnet = self.rnet = self.onet = None    # type: Any

        self.minsize = minimum_size_of_face # minimum size of face
        self.threshold = [0.6, 0.7, 0.7]    # three steps's threshold
        self.factor = 0.709                 # scale factor

    def init(self) -> None:
        """ doc """
        self.graph = tf.Graph()
        self.session = tf.Session(graph=self.graph)

        # pylint: disable=not-context-manager
        with self.graph.as_default():
            with self.session.as_default():
                self.pnet, self.rnet, self.onet = \
                    align.detect_face.create_mtcnn(self.session, None)

    def align(
            self,
            image: np.ndarray,
    ) -> Tuple[List[Any], List[Any]]:
        """ doc """

        bounding_boxes, landmarks = align.detect_face.detect_face(
            image,   # get rid of alpha channel(if any)
            self.minsize,
            self.pnet,
            self.rnet,
            self.onet,
            self.threshold,
            self.factor,
        )

        return bounding_boxes.tolist(), landmarks.tolist()

In [5]:
class FacenetBridge(object):
    """
    Bridge of Facenet
    """
    FACENET_MODEL = None   # type: str

    def __init__(self) -> None:
        self.graph = self.session = None        # type: Any

        self.placeholder_input = None           # type: Any
        self.placeholder_phase_train = None     # type: Any
        self.placeholder_embeddings = None      # type: Any

        self.FACENET_MODEL = FacenetBridge.get_model_path()

    def init(self) -> None:
        """ doc """
        self.graph = tf.Graph()
        self.session = tf.Session(graph=self.graph)

        # pylint: disable=not-context-manager
        with self.graph.as_default():
            with self.session.as_default():
                model_dir = os.path.expanduser(self.FACENET_MODEL)
                meta_file, ckpt_file = facenet.get_model_filenames(model_dir)
                saver = tf.train.import_meta_graph(
                    os.path.join(model_dir, meta_file),
                )
                saver.restore(
                    tf.get_default_session(),
                    os.path.join(model_dir, ckpt_file),
                )
                # facenet.load_model(self.FACENET_MODEL)

        self.placeholder_input = self.graph.get_tensor_by_name('input:0')
        self.placeholder_phase_train = \
            self.graph.get_tensor_by_name('phase_train:0')
        self.placeholder_embeddings = \
            self.graph.get_tensor_by_name('embeddings:0')

    @staticmethod
    def get_model_path() -> str:
        return model_path    
    
    def embeddings(
            self,
            images: np.ndarray
    ) -> np.ndarray:
        """
        Get embeddings
        """

        slices = np.empty((len(images), 160, 160, 3))
        for i in range(len(images)):
            slices[i,:,:,:] = facenet.prewhiten(images[i])
            
        feed_dict = {
            self.placeholder_input:         slices,
            self.placeholder_phase_train:   False,
        }
        # Use the facenet model to calcualte embeddings
        embeddings = self.session.run(
            self.placeholder_embeddings,
            feed_dict=feed_dict,
        )

        print("images " + str(len(images)))
        print("emb " + str(len(embeddings)))
        
        # Return the only row
        return embeddings

In [6]:
class MtcnnMain(threading.Thread):
    
    def __init__(self):
        threading.Thread.__init__(self)
        self.initNN()

    def initNN(self):
        self.mtcnn = MtcnnBridge()
        self.mtcnn.init()

    def run(self):
        print("Mtcnn thread: run")
        global frames
        global faces
        
        while True:
            if(not frames.empty()):
                frame = frames.get()                
                areas = self.mtcnn.align(frame[1])[0]                
                areasCount = len(areas)
                
                if (not areasCount == 0):
                    fullSize = frame[0]

                    height = np.size(fullSize, 0)
                    width = np.size(fullSize, 1)

                    positions = np.empty((areasCount, 4))
                    embeddings = np.empty((areasCount * 10, 160, 160, 3))

                    for i in range(areasCount):
                        pos = areas[i]
                        x = max(int(pos[0] * 2), 0)
                        y = max(int(pos[1] * 2), 0)
                        w = min(int(pos[2] * 2), width)
                        h = min(int(pos[3] * 2), height)

                        img = fullSize[y:h, x:w, 0:3]
                        img = cv2.resize(img, dsize=(160, 160), interpolation=cv2.INTER_CUBIC)                                                            

                        positions[i,:] = [x,y,w,h]

                        for j in range(10):
                            embeddings[i * 10 + j,:,:,:] = img
                        

                    if (not faces.full()):
                        faces.put([positions, embeddings])
            else:
                print("Mtcnn Idle")
                time.sleep(0.05)
                    

In [7]:
class FacenetMain(threading.Thread):
    
    def __init__(self):
        threading.Thread.__init__(self)
        self.user = []
        self.distances = []        
        self.loadUsers()
        self.initNN()

    def loadUsers(self):
        dataFile = "./data.json"
        file = open(dataFile, "r")
        self.users = json.loads(file.read())
        
    def initNN(self):
        self.face = FacenetBridge()
        self.face.init()

    def trackSessions(self, emb):
        nearDistances = []
        for user in self.users:
            userEmb = user['embedding']        
            userName = user['name']

            dist = distance.euclidean(userEmb, emb)
            if dist < 1.0:
                nearDistances.append((userName, dist))
                
        if(len(nearDistances) > 0):
            newlist = sorted(nearDistances, key=lambda x: x[1]) 

            print(newlist[0][0] + " - " + str(newlist[0][1]))
            
            self.distances.append(newlist[0][1])
        
        
    def run(self):
        print("Facenet thread: run")
        
        global faces
        
        while True:
            if(not faces.empty()):
                facesOnScreen = faces.get()                
                positions = facesOnScreen[0]
                images = facesOnScreen[1]
                                
                embeddings = self.face.embeddings(images)
                
                for emb in embeddings:                    
                    self.trackSessions(emb)                
                                        
            else:
                print("Facenet Idle")
                time.sleep(0.05)


In [8]:
class ImageGrabber(threading.Thread):
    def __init__(self, ID):
        threading.Thread.__init__(self)
        self.ID=ID
        self.cam=cv2.VideoCapture(ID)

    def run(self):
        print("Image grabber thread: run")
        
        global frames
        
        while True:
            ret, frame=self.cam.read()
            if (not frames.full()):
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                height = np.size(frame, 0)
                width = np.size(frame, 1)
                frame2 = cv2.resize(frame, dsize=(int(width / 2), int(height / 2)), interpolation=cv2.INTER_CUBIC)
                
                frames.put([frame, frame2])

            else:
                print('Grabber Idle')
                time.sleep(0.01)
                
            

In [ ]:
mtcnnCount = 2
facenetCount = 2

mtcnnInstances = []
facenetInstances = []

grabberInstance = ImageGrabber(0)
#grabber = ImageGrabber("rtsp://guest:sup3rGu3st@@10.10.101.156/ch1_1.h264")

for i in range(mtcnnCount):
    mtcnnInstances.append(MtcnnMain())
    
for i in range(facenetCount):
    facenetInstances.append(FacenetMain())

grabberInstance.start()

for mtcnn in mtcnnInstances:
    mtcnn.start()

for fn in facenetInstances:
    fn.start()
    fn.join()

for mtcnn in mtcnnInstances:
    mtcnn.join()

grabberInstance.join()

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from D:/NNA/node-facenet/models/model-20170512-110547.ckpt-250000
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from D:/NNA/node-facenet/models/model-20170512-110547.ckpt-250000
Image grabber thread: run
Mtcnn thread: run
Mtcnn thread: run
Facenet thread: run
Mtcnn Idle
Mtcnn Idle
Facenet Idle
Facenet Idle
Facenet Idle
Facenet Idle
Facenet Idle
Facenet Idle
Facenet Idle
Facenet Idle
Facenet Idle
Facenet Idle
Facenet Idle
Facenet Idle
Facenet Idle
Facenet Idle
Facenet Idle
Facenet Idle
Grabber Idle
Facenet Idle
Grabber Idle
Facenet Idle
Grabber Idle
Grabber Idle
Facenet Idle
Grabber Idle
Facenet Idle
Grabber Idle
Facenet Idle
Facenet Idle
Grabber Idle
Facenet Idle
Grabber Idle
Grabber Idle
Facenet Idle
Mtcnn Idle
Mtcnn Idle
Mtcnn Idle
Mtcnn Idle
Mtcnn Idle
Mtcnn Idle
Mtc

images 10
emb 10
Oleksii Makarov - 0.7896792586298319
Oleksii Makarov - 0.7896792586298319
Oleksii Makarov - 0.7896792586298319
Oleksii Makarov - 0.7896792586298319
Oleksii Makarov - 0.7896792586298319
Oleksii Makarov - 0.7896792586298319
Oleksii Makarov - 0.7896792586298319
Oleksii Makarov - 0.7896792586298319
Oleksii Makarov - 0.7896792969617481
Oleksii Makarov - 0.7896792969617481
Grabber Idle
images 10
emb 10
Oleksii Makarov - 0.7718463260949635
Oleksii Makarov - 0.7718463260949635
Oleksii Makarov - 0.7718463260949635
Oleksii Makarov - 0.7718463260949635
Oleksii Makarov - 0.7718463260949635
Oleksii Makarov - 0.7718463260949635
Oleksii Makarov - 0.7718463260949635
Oleksii Makarov - 0.7718463260949635
Oleksii Makarov - 0.7718462911893607
Oleksii Makarov - 0.7718462911893607
Grabber Idle
images 10
emb 10
Oleksii Makarov - 0.7490224969458594
Oleksii Makarov - 0.7490224969458594
Oleksii Makarov - 0.7490224969458594
Oleksii Makarov - 0.7490224969458594
Oleksii Makarov - 0.749022496945859

images 10
emb 10
Oleksii Makarov - 0.8054559049102998
Oleksii Makarov - 0.8054559049102998
Oleksii Makarov - 0.8054559049102998
Oleksii Makarov - 0.8054559049102998
Oleksii Makarov - 0.8054559049102998
Oleksii Makarov - 0.8054559049102998
Oleksii Makarov - 0.8054559049102998
Oleksii Makarov - 0.8054559049102998
Grabber IdleOleksii Makarov - 0.8054559049102998

Oleksii Makarov - 0.8054559049102998
Grabber Idle
Mtcnn Idle
images 10
emb 10
Oleksii Makarov - 0.8309834975419161
Oleksii Makarov - 0.8309834975419161
Oleksii Makarov - 0.8309834975419161
Oleksii Makarov - 0.8309834975419161
Oleksii Makarov - 0.8309834975419161
Oleksii Makarov - 0.8309834975419161
Oleksii Makarov - 0.8309834975419161
Oleksii Makarov - 0.8309834975419161
Oleksii Makarov - 0.8309834975419161
Oleksii Makarov - 0.8309834975419161
Grabber Idle
images 10
emb 10
Oleksii Makarov - 0.7585671159007191
Oleksii Makarov - 0.7585671159007191
Oleksii Makarov - 0.7585671159007191
Oleksii Makarov - 0.7585671159007191
Oleksii Mak

images 10
emb 10
Oleksii Makarov - 0.9697998260487436
Oleksii Makarov - 0.9697998260487436
Oleksii Makarov - 0.9697998260487436
Oleksii Makarov - 0.9697998260487436
Oleksii Makarov - 0.9697998260487436
Oleksii Makarov - 0.9697998260487436
Oleksii Makarov - 0.9697998260487436
Oleksii Makarov - 0.9697998260487436
Grabber Idle
Oleksii Makarov - 0.9697998260487436
Oleksii Makarov - 0.9697998260487436
images 10
emb 10
Oleksii Makarov - 0.8919852140751441
Oleksii Makarov - 0.8919852140751441
Oleksii Makarov - 0.8919852140751441
Oleksii Makarov - 0.8919852140751441
Oleksii Makarov - 0.8919852140751441
Oleksii Makarov - 0.8919852140751441
Oleksii Makarov - 0.8919852140751441
Oleksii Makarov - 0.8919852140751441
Oleksii Makarov - 0.8919852140751441
Oleksii Makarov - 0.8919852140751441
images 10
emb 10
Oleksii Makarov - 0.9215540440007159
Oleksii Makarov - 0.9215540440007159
Oleksii Makarov - 0.9215540440007159
Oleksii Makarov - 0.9215540440007159
Oleksii Makarov - 0.9215540440007159
Oleksii Mak

images 10
emb 10
Oleksii Makarov - 0.9046862003532068
Oleksii Makarov - 0.9046862003532068
Oleksii Makarov - 0.9046862003532068
Oleksii Makarov - 0.9046862003532068
Oleksii Makarov - 0.9046862003532068
Oleksii Makarov - 0.9046862003532068
Oleksii Makarov - 0.9046862003532068
Oleksii Makarov - 0.9046862003532068
Oleksii Makarov - 0.9046862003532068
Oleksii Makarov - 0.9046862003532068
images 10
emb 10
Oleksii Makarov - 0.8674228608587405
Oleksii Makarov - 0.8674228608587405
Oleksii Makarov - 0.8674228608587405
Oleksii Makarov - 0.8674228608587405
Oleksii Makarov - 0.8674228608587405
Oleksii Makarov - 0.8674228608587405
Oleksii Makarov - 0.8674228608587405
Oleksii Makarov - 0.8674228608587405
Oleksii Makarov - 0.8674229001706151Grabber Idle
Oleksii Makarov - 0.8674229001706151

images 10
emb 10
Oleksii Makarov - 0.8613001090627641
Oleksii Makarov - 0.8613001090627641
Oleksii Makarov - 0.8613001090627641
Oleksii Makarov - 0.8613001090627641
Oleksii Makarov - 0.8613001090627641
Oleksii Mak

...

images 10
emb 10
Oleksii Makarov - 0.7771193024066666
Oleksii Makarov - 0.7771193024066666
Oleksii Makarov - 0.7771193024066666
Oleksii Makarov - 0.7771193024066666
Oleksii Makarov - 0.7771193024066666
Oleksii Makarov - 0.7771193024066666
Oleksii Makarov - 0.7771193024066666
Oleksii Makarov - 0.7771193024066666
Oleksii Makarov - 0.7771193024066666
Grabber IdleOleksii Makarov - 0.7771193024066666

images 10
emb 10
Oleksii Makarov - 0.7500485927190158
Oleksii Makarov - 0.7500485927190158
Oleksii Makarov - 0.7500485927190158
Oleksii Makarov - 0.7500485927190158
Oleksii Makarov - 0.7500485927190158
Oleksii Makarov - 0.7500485927190158
Oleksii Makarov - 0.7500485927190158
Oleksii Makarov - 0.7500485927190158
Oleksii Makarov - 0.7500485927190158
Oleksii Makarov - 0.7500485927190158
Grabber Idle
images 10
emb 10
Oleksii Makarov - 0.7734998084011989
Oleksii Makarov - 0.7734998084011989
Oleksii Makarov - 0.7734998084011989
Oleksii Makarov - 0.7734998084011989
Oleksii Makarov - 0.773499808401198

images 10
emb 10
Oleksii Makarov - 0.6622226774151837
Oleksii Makarov - 0.6622226774151837
Oleksii Makarov - 0.6622226774151837
Oleksii Makarov - 0.6622226774151837
Oleksii Makarov - 0.6622226774151837
Oleksii Makarov - 0.6622226774151837
Oleksii Makarov - 0.6622226774151837
Oleksii Makarov - 0.6622226774151837
Oleksii Makarov - 0.6622226522299262
Oleksii Makarov - 0.6622226522299262
Grabber Idle
images 10
emb 10
Oleksii Makarov - 0.6714748475130674
Oleksii Makarov - 0.6714748475130674
Oleksii Makarov - 0.6714748475130674
Oleksii Makarov - 0.6714748475130674
Oleksii Makarov - 0.6714748475130674
Oleksii Makarov - 0.6714748475130674
Oleksii Makarov - 0.6714748475130674
Oleksii Makarov - 0.6714748475130674
Oleksii Makarov - 0.6714748790147439
Oleksii Makarov - 0.6714748790147439
Grabber Idle
images 10
emb 10
Oleksii Makarov - 0.6842905069007664
Oleksii Makarov - 0.6842905069007664
Oleksii Makarov - 0.6842905069007664
Oleksii Makarov - 0.6842905069007664
Oleksii Makarov - 0.684290506900766

images 10
emb 10
Oleksii Makarov - 0.6767307459548817
Oleksii Makarov - 0.6767307459548817
Oleksii Makarov - 0.6767307459548817
Oleksii Makarov - 0.6767307459548817
Oleksii Makarov - 0.6767307459548817
Oleksii Makarov - 0.6767307459548817
Oleksii Makarov - 0.6767307459548817
Oleksii Makarov - 0.6767307459548817
Oleksii Makarov - 0.6767307459548817
Oleksii Makarov - 0.6767307459548817
images 10
emb 10
Oleksii Makarov - 0.7056149219991864
Oleksii Makarov - 0.7056149219991864
Oleksii Makarov - 0.7056149219991864
Oleksii Makarov - 0.7056149219991864
Oleksii Makarov - 0.7056149219991864
Oleksii Makarov - 0.7056149219991864
Oleksii Makarov - 0.7056149219991864
Oleksii Makarov - 0.7056149219991864
Oleksii Makarov - 0.7056148896602757
Oleksii Makarov - 0.7056148896602757
images 10
emb 10
Oleksii Makarov - 0.6912016317195395
Oleksii Makarov - 0.6912016317195395
Oleksii Makarov - 0.6912016317195395
Oleksii Makarov - 0.6912016317195395
Oleksii Makarov - 0.6912016317195395
Oleksii Makarov - 0.6912

Mtcnn Idle
images 10
emb 10
Oleksii Makarov - 0.8150435578884156
Oleksii Makarov - 0.8150435578884156
Oleksii Makarov - 0.8150435578884156
Oleksii Makarov - 0.8150435578884156
Oleksii Makarov - 0.8150435578884156
Oleksii Makarov - 0.8150435578884156
Oleksii Makarov - 0.8150435578884156
Oleksii Makarov - 0.8150435578884156
Oleksii Makarov - 0.8150435578884156Grabber Idle

Oleksii Makarov - 0.8150435578884156
images 10
emb 10
Oleksii Makarov - 0.8528963882979598
Oleksii Makarov - 0.8528963882979598
Oleksii Makarov - 0.8528963882979598
Oleksii Makarov - 0.8528963882979598
Oleksii Makarov - 0.8528963882979598
Oleksii Makarov - 0.8528963882979598
Oleksii Makarov - 0.8528963882979598
Oleksii Makarov - 0.8528963882979598
Grabber Idle
Oleksii Makarov - 0.8528963882979598
Oleksii Makarov - 0.8528963882979598
images 10
emb 10
Oleksii Makarov - 0.8189123071974946
Oleksii Makarov - 0.8189123071974946
Oleksii Makarov - 0.8189123071974946
Oleksii Makarov - 0.8189123071974946
Oleksii Makarov - 0.8189

Grabber Idle
images 10
emb 10
Oleksii Makarov - 0.8307667158052294
Oleksii Makarov - 0.8307667158052294
Oleksii Makarov - 0.8307667158052294
Oleksii Makarov - 0.8307667158052294
Oleksii Makarov - 0.8307667158052294
Oleksii Makarov - 0.8307667158052294
Oleksii Makarov - 0.8307667158052294
Oleksii Makarov - 0.8307667158052294
Grabber IdleOleksii Makarov - 0.8307667158052294

Oleksii Makarov - 0.8307667158052294
images 10
emb 10
Oleksii Makarov - 0.7952646432315076
Oleksii Makarov - 0.7952646432315076
Oleksii Makarov - 0.7952646432315076
Oleksii Makarov - 0.7952646432315076
Oleksii Makarov - 0.7952646432315076
Oleksii Makarov - 0.7952646432315076
Oleksii Makarov - 0.7952646432315076
Grabber Idle
Oleksii Makarov - 0.7952646432315076
Oleksii Makarov - 0.7952646028705365
Oleksii Makarov - 0.7952646028705365
Grabber Idle
images 10
emb 10
Oleksii Makarov - 0.7347867541013354
Oleksii Makarov - 0.7347867541013354
Oleksii Makarov - 0.7347867541013354
Oleksii Makarov - 0.7347867541013354
Oleksii M